<a href="https://colab.research.google.com/github/highshuang/biostat_deep_learning_spring_2021/blob/develop/Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alexnet Implementation on Beans Dataset 

####Step 1: Import libraries and Beans data 

In [96]:
import os 
import time
import keras
# Import TensorFlow and TensorFlow Datasets
#Download the dataset and divide it into train and test
datasets, info = tfds.load('beans', with_info=True, as_supervised=True)
train_ds, test_ds, val_ds = datasets['train'], datasets['test'], datasets['validation']

# show the information of dataset 
info

tfds.core.DatasetInfo(
    name='beans',
    version=0.1.0,
    description='Beans is a dataset of images of beans taken in the field using smartphone
cameras. It consists of 3 classes: 2 disease classes and the healthy class.
Diseases depicted include Angular Leaf Spot and Bean Rust. Data was annotated
by experts from the National Crops Resources Research Institute (NaCRRI) in
Uganda and collected by the Makerere AI research lab.',
    homepage='https://github.com/AI-Lab-Makerere/ibean/',
    features=FeaturesDict({
        'image': Image(shape=(500, 500, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=1295,
    splits={
        'test': 128,
        'train': 1034,
        'validation': 133,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {beansdata,
        author="Makerere AI Lab",
        title="Bean disease dataset",
        month="January",
        year="2020",
        url="https://git

#### Step2: Resize image data to 227x227 

In [97]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.0
  image = tf.image.resize(image, [227, 227])
  return image, label

train_ds = (train_ds.map(scale)
                    .shuffle(buffer_size=1034)
                    .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds.map(scale)
                    .shuffle(buffer_size=128)
                    .batch(batch_size=32, drop_remainder=True))
val_ds = (val_ds.map(scale)
                    .shuffle(buffer_size=133)
                    .batch(batch_size=32, drop_remainder=True))

val_ds

<BatchDataset shapes: ((32, 227, 227, 3), (32,)), types: (tf.float32, tf.int64)>

Step 3: Build Alexnet construction with keras 

In [89]:
alexnet = keras.models.Sequential([
# 1st Convolutional Layer
keras.layers.Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4),  activation = "relu"),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),

# 2nd Convolutional Layer
keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same", activation = "relu"),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),

# 3rd Convolutional Layer
keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"),
keras.layers.BatchNormalization(),

# 4th Convolutional Layer
keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"),
keras.layers.BatchNormalization(),

# 5th Convolutional Layer
keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2)),

# Passing it to a Fully Connected layer
keras.layers.Flatten(),
# 1st Fully Connected Layer
keras.layers.Dense(units = 9216, activation = "relu"),
keras.layers.Dropout(0.4),

# 2nd Fully Connected Layer
keras.layers.Dense(units = 4096, activation = "relu"),
keras.layers.Dropout(0.4),

# 3rd Fully Connected Layer
keras.layers.Dense(4096, activation = "relu"),
keras.layers.Dropout(0.4),

# Output Layer
keras.layers.Dense(3, activation = "softmax") #As we have three classes
])

alexnet.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.optimizers.SGD(lr=0.01), metrics=['accuracy'])

alexnet.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_20 (Batc (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_21 (Batc (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 13, 13, 384)      

#### Step 5: Train Alexnet with beans_train and beans_Validation

In [90]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [75]:
alexnet.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])

In [98]:
import numpy as np 
np.random.seed(10)
alexnet.fit(train_ds, epochs=15, validation_data=val_ds, validation_freq=1, callbacks=[tensorboard_cb])

Epoch 1/15
32/32 [==============================] - 185s 6s/step - loss: 0.8166 - accuracy: 0.6826 - val_loss: 1.1976 - val_accuracy: 0.3203
Epoch 2/15
32/32 [==============================] - 183s 6s/step - loss: 0.6699 - accuracy: 0.7373 - val_loss: 1.3504 - val_accuracy: 0.3359
Epoch 3/15
32/32 [==============================] - 191s 6s/step - loss: 0.6452 - accuracy: 0.7305 - val_loss: 1.4620 - val_accuracy: 0.3359
Epoch 4/15
32/32 [==============================] - 182s 6s/step - loss: 0.6260 - accuracy: 0.7461 - val_loss: 1.6060 - val_accuracy: 0.3828
Epoch 5/15
32/32 [==============================] - 183s 6s/step - loss: 0.5180 - accuracy: 0.7910 - val_loss: 1.3500 - val_accuracy: 0.4453
Epoch 6/15
32/32 [==============================] - 183s 6s/step - loss: 0.4500 - accuracy: 0.8164 - val_loss: 1.6360 - val_accuracy: 0.4141
Epoch 7/15
32/32 [==============================] - 185s 6s/step - loss: 0.3912 - accuracy: 0.8496 - val_loss: 1.0710 - val_accuracy: 0.5234
Epoch 8/15
32

In [99]:
alexnet.evaluate(test_ds)

4/4 [==============================] - 6s 1s/step - loss: 0.5413 - accuracy: 0.8516


[0.5413299202919006, 0.8515625]